In [2]:

# Set up notebook
%pprint
import sys
if (osp.join(os.pardir, 'py') not in sys.path): sys.path.insert(1, osp.join(os.pardir, 'py'))

Pretty printing has been turned OFF


In [3]:

# load libraries
from FRVRS import (nu, fu, display, osp, read_excel)
from pandas import get_dummies
from re import split, search, sub, MULTILINE
from scipy.stats import f_oneway, ttest_ind, kruskal, norm
import itertools


### Add a column "HC_completed" where you provide a 1 for hemorrage control completed for that responder for the scene and a 0 for those that didnt

In [6]:

# load data frames
data_frames_dict = nu.load_data_frames(
    first_responder_master_registry_df='', first_responder_master_registry_file_stats_df='',
    first_responder_master_registry_scene_stats_df='', count_swaps_df='',
    verbose=False
)
triage_paper_df = data_frames_dict['first_responder_master_registry_df']
print('triage_paper_df', triage_paper_df.shape[0], triage_paper_df.session_uuid.nunique(), triage_paper_df.shape[1]) # 367627 290 111
file_stats_df = data_frames_dict['first_responder_master_registry_file_stats_df']
print('file_stats_df', file_stats_df.shape[0], file_stats_df.file_name.nunique(), file_stats_df.shape[1]) # 290 290 19
scene_stats_df = data_frames_dict['first_responder_master_registry_scene_stats_df']
print('scene_stats_df', scene_stats_df.shape[0], scene_stats_df.session_uuid.nunique(), scene_stats_df.shape[1]) # 420 290 48
count_swaps_df = data_frames_dict['count_swaps_df']
print('count_swaps_df', count_swaps_df.shape[0], count_swaps_df.session_uuid.nunique(), count_swaps_df.shape[1]) # 289 289 3

triage_paper_df 367627 290 111
file_stats_df 290 290 19
scene_stats_df 420 290 48
count_swaps_df 315 289 3


In [7]:

groups_list = ['Paramedic', 'EMT-Basic', 'EM Resident', 'Medical Student']
patient_count_filter_fn = lambda scene_df: True
nongrouped_patients_list = nu.load_object('nongrouped_patients_list')

In [8]:

# Merge only count swaps scenes
elevens_df = fu.get_elevens_dataframe(
    triage_paper_df,
    file_stats_df,
    scene_stats_df,
    needed_columns=[
        'scene_type', 'is_scene_aborted', 'is_a_one_triage_file', 'responder_category', 'responder_type', 'overall_category',
        'file_name', 'session_file_name', 'encounter_layout'
    ],
    patient_count_filter_fn=None
)
print('elevens_df', elevens_df.shape[0], elevens_df.session_uuid.nunique(), elevens_df.shape[1]) # 397230 331 121

# Get a dataset of only first11 patients
mask_series = ~elevens_df.patient_id.isin(nongrouped_patients_list)
elevens_df = elevens_df[mask_series]
print('elevens_df', elevens_df.shape[0], elevens_df.session_uuid.nunique(), elevens_df.shape[1]) # 390108 331 121

# Get a dataset of only "First 11" encounter layouts
mask_series = (elevens_df.encounter_layout == 'DCEMS 11')
display(elevens_df.groupby('encounter_layout').size().to_frame().rename(columns={0: 'record_count'}).sort_values(
    'record_count', ascending=False
).head(5))
elevens_df = elevens_df[~mask_series]
print('elevens_df', elevens_df.shape[0], elevens_df.session_uuid.nunique(), elevens_df.shape[1]) # 367025 295 119

mask_series = elevens_df.responder_category.isin(groups_list)
elevens_df = elevens_df[mask_series]
print('elevens_df', elevens_df.shape[0], elevens_df.session_uuid.nunique(), elevens_df.shape[1]) # 367025 295 119

elevens_df 365543 289 120
elevens_df 365521 289 120


,record_count
encounter_layout,
First 11,365238


elevens_df 365521 289 120
elevens_df 358586 282 120


In [29]:

file_name = 'swaps_walk_wave_anova_df20240516.xlsx'
file_path = osp.join(nu.data_folder, 'xlsx', file_name)
swaps_walk_wave_df = read_excel(file_path)
print(swaps_walk_wave_df.shape)
for (session_uuid, scene_id), idx_df in swaps_walk_wave_df.groupby(fu.scene_groupby_columns):

    # Get the whole scene history
    mask_series = True
    for cn in fu.scene_groupby_columns: mask_series &= (triage_paper_df[cn] == eval(cn))
    if mask_series.any():
        scene_df = triage_paper_df[mask_series]
        swaps_walk_wave_df.loc[idx_df.index, 'HC_completed'] = int(fu.get_percent_hemorrhage_controlled(scene_df) == 100)
    else:
        display(idx_df.T)
        break

(291, 17)


In [30]:

nu.save_data_frames(swaps_walk_wave_df20240520=swaps_walk_wave_df, verbose=True)

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/swaps_walk_wave_df20240520.csv
